<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Utility" data-toc-modified-id="Utility-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Utility</a></span></li><li><span><a href="#Feature-Scaling" data-toc-modified-id="Feature-Scaling-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Feature Scaling</a></span></li></ul></li><li><span><a href="#Model" data-toc-modified-id="Model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Model</a></span><ul class="toc-item"><li><span><a href="#one-vs-rest" data-toc-modified-id="one-vs-rest-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>one-vs-rest</a></span></li><li><span><a href="#radius_neighbors-Approach" data-toc-modified-id="radius_neighbors-Approach-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>radius_neighbors Approach</a></span><ul class="toc-item"><li><span><a href="#Multiple-products" data-toc-modified-id="Multiple-products-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Multiple products</a></span></li><li><span><a href="#All-products" data-toc-modified-id="All-products-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>All products</a></span></li></ul></li><li><span><a href="#kneighbors-approach" data-toc-modified-id="kneighbors-approach-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>kneighbors approach</a></span><ul class="toc-item"><li><span><a href="#Single-product" data-toc-modified-id="Single-product-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>Single product</a></span></li><li><span><a href="#Multiple-products" data-toc-modified-id="Multiple-products-2.3.2"><span class="toc-item-num">2.3.2&nbsp;&nbsp;</span>Multiple products</a></span></li><li><span><a href="#All-products" data-toc-modified-id="All-products-2.3.3"><span class="toc-item-num">2.3.3&nbsp;&nbsp;</span>All products</a></span></li></ul></li></ul></li></ul></div>

# Setup

In [ ]:
!/home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip -V

In [50]:
# setup to see the execution time in each cell

!/home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip install ipython-autotime
!/home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip install directory_structure
!/home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip install handcalcs
!/home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip install pytz
!/home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip install skimage
# !/home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip install opencv-contrib-python==4.4.0.44
# %load_ext autotime

/bin/bash: line 1: /home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip: Is a directory
/bin/bash: line 1: /home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip: Is a directory
/bin/bash: line 1: /home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip: Is a directory
/bin/bash: line 1: /home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip: Is a directory
/bin/bash: line 1: /home/hui/anaconda3/envs/ml/lib/python3.10/site-packages/pip: Is a directory


In [1]:
import os
import random
import glob
import cv2
import shutil
import pickle
import PIL
import pandas as pd
from PIL import Image
import numpy as np
from numpy import mean
import matplotlib.pyplot as plt
import plotly.express as px

from skimage.feature import local_binary_pattern
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# import cupy as cp

# from google.colab.patches import cv2_imshow
# from directory_structure import Tree

In [2]:
print (cv2.__version__)

4.5.5


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# replace the dir if needed

ROOT_DIR = "/home/hui/Projects/g5/datasets"  # Henry's dir
# ROOT_DIR = "/content/drive/MyDrive/Project - G5/Dataset/"    #Lejia's dir
# ROOT_DIR = "/content/drive/MyDrive/Project - G5/Dataset/" #Nan's dir
# ROOT_DIR = "/content/drive/MyDrive/Project - G5/Dataset"


# path for image augmentation
RESIZE_DIR = ROOT_DIR + "/augmentation/resize"


# path for the different handcraft features
RAW_PIXEL_DIR = ROOT_DIR + "/handcraft_features/raw_pixel"
SIFT_DIR = ROOT_DIR + "/handcraft_features/sift"
HOG_DIR = ROOT_DIR + "/handcraft_features/hog"
LBP_DIR = ROOT_DIR + "/handcraft_features/lbp"


# menu datasets
MENU_FOOD = ROOT_DIR + "/MeetFresh_menu_food_2.csv"
MENU_DRINK = ROOT_DIR + "/MeetFresh_menu_drink_2.csv"


# combination of p and r values for LBP
LBP_CONFIG = [
  {"p": 8, "r": 1},
  {"p": 16, "r": 2},
  {"p": 24, "r": 3},
]

## Utility

Retrieve local descriptor

In [6]:
def retrieve_stored_features(path: str, product_name: str):
    # retrieve a stored local descriptor based on given name
    try:
        file_path = path + "/" + product_name + ".txt"
        file = open(file_path, 'rb')
        descriptor = pickle.load(file)
        file.close()

        return descriptor

    except FileNotFoundError:
        print("Wrong file name or path for {}".format(product_name))
        return None
    except e:
        print("Error due to: {}".format(e))
        return None

In [14]:
x = {
    "8_1": list(),
    "16_2": list(),
    "24_3": list(),
}

for pr in LBP_CONFIG:
    
    files = os.listdir(LBP_DIR + "/" + str(pr["p"]) + "_" + str(pr["r"]))
    
    for file in files:
        x[str(pr["p"]) + "_" + str(pr["r"])].append(retrieve_stored_features(LBP_DIR + "/" + str(pr["p"]) + "_" + str(pr["r"]), file.split(".")[0]))

print(">>> All jobs done!")

>>> All jobs done!


In [15]:
x

{'8_1': [array([0.03697205, 0.04345703, 0.03501892, 0.08866882, 0.17086792,
         0.10639954, 0.04229736, 0.05725098, 0.34178162, 0.07728577]),
  array([0.01309204, 0.01733398, 0.01712036, 0.04484558, 0.07510376,
         0.0615387 , 0.01977539, 0.02305603, 0.70056152, 0.02757263]),
  array([0.01768494, 0.02029419, 0.01124573, 0.0294342 , 0.04870605,
         0.05683899, 0.01506042, 0.02661133, 0.73225403, 0.04187012]),
  array([0.03819275, 0.05169678, 0.03582764, 0.07725525, 0.11869812,
         0.11116028, 0.05270386, 0.05810547, 0.35917664, 0.09718323]),
  array([0.0227356 , 0.0375061 , 0.02339172, 0.06677246, 0.12666321,
         0.10916138, 0.03314209, 0.04330444, 0.4743042 , 0.0630188 ]),
  array([0.01266479, 0.02940369, 0.02804565, 0.10467529, 0.17068481,
         0.18739319, 0.07232666, 0.04608154, 0.29408264, 0.05464172]),
  array([0.01875305, 0.0405426 , 0.01589966, 0.08781433, 0.16439819,
         0.16546631, 0.04893494, 0.068573  , 0.30809021, 0.08152771]),
  array([0.01

## Feature Scaling

Feature Scaling before fitting the data into the model.

In [14]:
for pr in LBP_CONFIG:
    sc = StandardScaler()
    x = sc.fit_transform(x)

# Model

In [15]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute')

In [16]:
index = 0

In [17]:
model_knn.fit(x)

NearestNeighbors(algorithm='brute', metric='cosine')

In [56]:
dir(model_knn)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_algorithm_metric',
 '_check_feature_names',
 '_check_n_features',
 '_fit',
 '_fit_X',
 '_fit_method',
 '_get_param_names',
 '_get_tags',
 '_kneighbors_reduce_func',
 '_more_tags',
 '_radius_neighbors_reduce_func',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_tree',
 '_validate_data',
 'algorithm',
 'effective_metric_',
 'effective_metric_params_',
 'fit',
 'get_params',
 'kneighbors',
 'kneighbors_graph',
 'leaf_size',
 'metric',
 'metric_params',
 'n_features_in_',
 'n_jobs',
 'n_neighbors',
 'n_samples_fit_',
 'p',
 'radius'

In [64]:
model_knn.n_neighbors

5

## one-vs-rest

In [30]:
def transform2(x):
    sc = StandardScaler()
    x_ = sc.fit_transform(x)
    
    return x_

## radius_neighbors Approach

In [204]:
r = np.arange(0.1, 1.1, 0.1)

for i in r:
    test_product = transform2(retrieve_stored_features(RAW_PIXEL_DIR, "Black Sugar Boba MIlk Tea").reshape(1, -1))

    distances, indices = model_knn.radius_neighbors(test_product, radius=i)

    print("R: {}".format(i))
    print(distances, indices, end="\n\n")

    
    
    
    
    
# for i in range(len(indices)):
#     idx = np.where(distances[i,]<1.9)[i]
# #     print(files[idx])
#     print(idx)

R: 0.1
[array([], dtype=float64)] [array([], dtype=int64)]

R: 0.2
[array([], dtype=float64)] [array([], dtype=int64)]

R: 0.30000000000000004
[array([], dtype=float64)] [array([], dtype=int64)]

R: 0.4
[array([], dtype=float64)] [array([], dtype=int64)]

R: 0.5
[array([], dtype=float64)] [array([], dtype=int64)]

R: 0.6
[array([], dtype=float64)] [array([], dtype=int64)]

R: 0.7000000000000001
[array([], dtype=float64)] [array([], dtype=int64)]

R: 0.8
[array([], dtype=float64)] [array([], dtype=int64)]

R: 0.9
[array([], dtype=float64)] [array([], dtype=int64)]

R: 1.0
[array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1.])          

### Multiple products

In [169]:
distances, indices = model_knn.radius_neighbors(x[3:9], radius=0.1)

print("Distance: ", end="\n")
print(distances, end="\n\n")
print("Indices: ")
print(indices)

# for i in range(len(distances)):
#     idx = np.where(distances[i,]<0.8)[0] 
#     print(files[idx])
#     print(indices)
#     print(idx)

Distance: 
[array([1.53210777e-14]) array([0.]) array([2.33146835e-15])
 array([1.4321877e-14]) array([3.67483821e-14, 1.26507856e-02])
 array([0.])]

Indices: 
[array([3]) array([4]) array([5]) array([6]) array([ 7, 85]) array([8])]


### All products

In [164]:
distances, indices = model_knn.radius_neighbors(x, radius=0.1)

print(distances, indices)

# for i in range(len(distances)-1):
#     print(distances[i], indices[i])
#     if distances.flatten()[i] == 0:
#         print("Recommend: {} with distance {}".format(files[indices[0][i]].split(".")[0], distances.flatten()[i]))
#     else:
#         print("{}: \t {} \t\t with distance {}".format(i, files[indices[0][i]].split(".")[0], distances.flatten()[i]))

[array([0.]) array([0.]) array([0.        , 0.09553109]) array([0.])
 array([0.]) array([0.]) array([0.]) array([0.        , 0.01265079])
 array([0.]) array([0.]) array([0.]) array([0.]) array([0.]) array([0.])
 array([0.]) array([0.]) array([0.]) array([0.]) array([0.]) array([0.])
 array([0.]) array([0.        , 0.05661112])
 array([0.        , 0.06423271]) array([0.]) array([0.]) array([0.])
 array([0.]) array([0.]) array([0.]) array([0.]) array([0.]) array([0.])
 array([0.]) array([0.]) array([0.]) array([0.])
 array([0.05661112, 0.        ]) array([0.]) array([0.]) array([0.])
 array([0.]) array([0.]) array([0.]) array([0.]) array([0.]) array([0.])
 array([0.]) array([0.]) array([0.]) array([0.]) array([0.]) array([0.])
 array([0.]) array([0.]) array([0.]) array([0.        , 0.01477863])
 array([0.]) array([0.]) array([0.]) array([0.        , 0.09002082])
 array([0.]) array([0.]) array([0.]) array([0.]) array([0.]) array([0.])
 array([0.]) array([0.]) array([0.]) array([0.09002082

## kneighbors approach

### Single product

In [113]:
test_product = transform2(retrieve_stored_features(RAW_PIXEL_DIR, "Black Sugar Boba MIlk Tea").reshape(1, -1))

distances, indices = model_knn.kneighbors(test_product, n_neighbors=9)

for i in range(len(distances)):
#     print(distances[i], indices[i])
#     if i == 0:
#         print("Recommend: {} with distance {}".format(files[i].split(".")[0], distances.flatten()[i]))
#     else:
    print("{}: {} with distance {}".format(i, files[indices[0][i]].split(".")[0], distances.flatten()[i]))

0: Red Bean Almond Pudding with distance 1.0


### Multiple products

In [119]:
distances, indices = model_knn.kneighbors(x[3:9])

for i in range(len(distances)-1):
#     print(distances[i], indices[i])
#     if i == 0:
#         print("Recommend: {} with distance {}".format(files[i].split(".")[0], distances.flatten()[i]))
#     else:
#         print("{}: \t {} \t\t with distance {}".format(i, files[i].split(".")[0], distances.flatten()[i]))
    print("{}: {} with distance \t\t {}".format(i, files[indices[0][i]].split(".")[0], distances.flatten()[i]))

0: Pudding _ Q Mochi Milk Shaved Ice with distance 		 1.532107773982716e-14
1: Q Mochi Milk Shaved Ice with distance 		 0.25772779868722306
2: Strawberry Milk Shaved Ice with distance 		 0.45424211313098184
3: Red Bean Milk Shaved Ice with distance 		 0.5740652263182903
4: Strawberry Smoothie with distance 		 0.6026677620544814


### All products

In [125]:
distances, indices = model_knn.kneighbors(x)

for i in range(len(distances)-1):
    print(distances[i], indices[i])
#     if distances.flatten()[i] == 0:
#         print("Recommend: {} with distance {}".format(files[indices[0][i]].split(".")[0], distances.flatten()[i]))
#     else:
#         print("{}: \t {} \t\t with distance {}".format(i, files[indices[0][i]].split(".")[0], distances.flatten()[i]))

[0.         0.47086725 0.55221346 0.67114748 0.68603384] [ 0 52 90 43 62]
[0.         0.439676   0.45612076 0.53702196 0.65741782] [ 1 68 32 53 22]
[0.         0.09553109 0.10549497 0.10999224 0.11267963] [ 2 86 22 71 76]
[0.         0.2577278  0.45424211 0.57406523 0.60266776] [ 3 17 41 50 44]
[0.         0.4205541  0.4714414  0.7781678  0.82731105] [ 4 13 58 46 50]
[0.         0.49658358 0.55527263 0.64527162 0.6680044 ] [ 5 47 66 11 20]
[0.         0.60926289 0.62894601 0.63661872 0.72323505] [ 6 27 83 75 72]
[0.         0.01265079 0.19725428 0.26146803 0.27303828] [ 7 85 19 88 43]
[0.         0.1434542  0.19971846 0.24201931 0.27368774] [ 8 30 70 45 37]
[0.         0.55295966 0.63431483 0.68205877 0.70608582] [ 9 14 31 64 65]
[0.         0.5727443  0.58341849 0.65331404 0.65604736] [10 41 23 50 17]
[0.         0.37009044 0.3790708  0.38479476 0.39375839] [11 84 47 81 20]
[0.         0.26751945 0.28816801 0.31536333 0.32373524] [12 67 77 55 88]
[0.         0.36466598 0.4205541  0.69